In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from yolov2 import YOLOv2D19 as YOLOv2
from loss import YOLOv2Loss
from utils import IoU
from detection_datasets import VOCDatasetV2
from torch.profiler import profile, ProfilerActivity
import pickle
import datetime
with open('anchors_VOC0712trainval.pickle', 'rb') as handle:
    anchors = pickle.load(handle)

In [2]:
import sys
sys.path.insert(1, '../')
from data_preprocessing import get_norms

In [3]:
torch.cuda.empty_cache()

In [4]:
device = torch.device('cuda:0')
dtype = torch.float32

In [5]:
scaler = torch.amp.GradScaler("cuda")

In [6]:
model = YOLOv2(device=device, dtype=dtype)

C:\Me\PJAIT\Thesis\Code\yolov2-to-yolov8\Models\yolov2.py:138: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_dict_path, map_location=self.devic

In [7]:
norms = get_norms('../../datasets/VOCdevkit/trainval_norms.json')
means = norms['means']
stds = norms['stds']

In [8]:
transforms = A.Compose([
    A.Resize(width=416, height=416),
    A.VerticalFlip(p=1.0),
    A.Normalize(mean=means, std=stds),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc'))
train_set = VOCDatasetV2(devkit_path = '../../datasets/VOCdevkit/', scales=[13], anchors=anchors, transforms=transforms, 
                         dtype=dtype, device=device)

True ../../datasets/VOCdevkit/VOC2007\ImageSets\Main\trainval.txt
True ../../datasets/VOCdevkit/VOC2012\ImageSets\Main\trainval.txt


In [9]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=False)

My Loss

In [70]:
loss_fn = YOLOv2Loss(anchors=anchors)

In [71]:
torch.cuda.empty_cache()

In [72]:
scaler = torch.cuda.amp.GradScaler()
for img, label in train_loader:
    with torch.cuda.amp.autocast():
        out = model(img)
        loss = loss_fn(out, label)

        _datetime = datetime.datetime.now()
        print(f"{_datetime}")
        scaler.scale(loss).backward()
        _datetime = datetime.datetime.now()
        print(f"{_datetime}")
    break

C:\conda_tmp\ipykernel_10768\1383623768.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
C:\conda_tmp\ipykernel_10768\1383623768.py:3: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


2024-12-22 12:32:56.145343
2024-12-22 12:32:56.255371


In [73]:
loss

tensor(7507.0454, device='cuda:0', grad_fn=<AddBackward0>)

In [74]:
for img, label in train_loader:
    out = model(img)
    loss = loss_fn(out, label)

    _datetime = datetime.datetime.now()
    print(f"{_datetime}")
    loss.backward()
    _datetime = datetime.datetime.now()
    print(f"{_datetime}")
    break

2024-12-22 12:33:42.113198
2024-12-22 12:34:17.220017


In [75]:
loss

tensor(7526.1958, device='cuda:0', grad_fn=<AddBackward0>)